In [1]:
import daw_readout
import process_data
import pandas as pd
import numpy as np
import h5py
from tqdm import tqdm
import sys
import time
import os

In [2]:
#file_list = 'runlist/darkrate_monitor_with_GECO2020'
#file_list = 'runlist/LED_1.7V_11ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_7.5ns_0.5_ratio_runs'
#file_list = 'runlist/LED_1.7V_7.5ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_10ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_11ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_12ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_13ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_14ns_400_ratio_runs'
file_list = 'runlist/LED_1.7V_15ns_400_ratio_runs'

In [4]:
path_save =""
winfo =[]
with open(file_list, 'r') as list:
    for line in list:    
        rawfilename = line.rstrip('\n')[17 :] 
        rawdata = daw_readout.DAWDemoWaveParser(rawfilename)     
        for wave in tqdm(rawdata) :            
            ch = wave.Channel
            ttt = wave.Timestamp
            base = wave.Baseline
            pulse = wave.Waveform
            st, minp, ed = process_data.pusle_index(pulse)
            ht = base - wave.Waveform[minp]            
            area = process_data.pusle_area(pulse, st, ed, base)
            winfo.append({         
                #'EvtID': i,
                'Ch':ch,
                'TTT':ttt,   ## Trigger time tag
                'Baseline': base, 
                'Hight': ht, 
                'Area': area,     ## area of pulse cal by dynamic range
            })            
        file_tag = line.rstrip('\n')[17 :].rstrip('.bin')[24:][: -13]  
        path_save = "outnpy/{}.npy".format(file_tag)

361095it [00:08, 40158.35it/s]


In [11]:
winfo[:12]

[{'Ch': 0, 'TTT': 103988, 'Baseline': 15199, 'Hight': 8596, 'Area': 19973},
 {'Ch': 1, 'TTT': 103988, 'Baseline': 15339, 'Hight': 344, 'Area': 659},
 {'Ch': 2, 'TTT': 103988, 'Baseline': 8115, 'Hight': 10, 'Area': -6},
 {'Ch': 0, 'TTT': 353988, 'Baseline': 15200, 'Hight': 10258, 'Area': 20982},
 {'Ch': 1, 'TTT': 353988, 'Baseline': 15339, 'Hight': 208, 'Area': 313},
 {'Ch': 2, 'TTT': 353988, 'Baseline': 8116, 'Hight': 7, 'Area': -1},
 {'Ch': 1, 'TTT': 603988, 'Baseline': 15337, 'Hight': 187, 'Area': 368},
 {'Ch': 2, 'TTT': 603988, 'Baseline': 8117, 'Hight': 20, 'Area': 7},
 {'Ch': 0, 'TTT': 603988, 'Baseline': 15199, 'Hight': 11523, 'Area': 21662},
 {'Ch': 0, 'TTT': 853984, 'Baseline': 15200, 'Hight': 9971, 'Area': 19930},
 {'Ch': 1, 'TTT': 853984, 'Baseline': 15340, 'Hight': 609, 'Area': 1061},
 {'Ch': 2, 'TTT': 853984, 'Baseline': 8116, 'Hight': 17, 'Area': -10}]

In [13]:
path_save

'outnpy/LV2415_anodereadout_LV2414_dualreadout_20240821_LED_1.7V_15ns_400_ratio_run.h5df'

In [15]:
df = pd.DataFrame(winfo)

(361095, 5)

In [19]:
df[:12]

,Ch,TTT,Baseline,Hight,Area
0,0,103988,15199,8596,19973
1,1,103988,15339,344,659
2,2,103988,8115,10,-6
3,0,353988,15200,10258,20982
4,1,353988,15339,208,313
5,2,353988,8116,7,-1
6,1,603988,15337,187,368
7,2,603988,8117,20,7
8,0,603988,15199,11523,21662
9,0,853984,15200,9971,19930


In [17]:
data_array = df.values

In [18]:
np.save('outnpy/LV2415_anodereadout_LV2414_dualreadout_20240821_LED_1.7V_15ns_400_ratio_run.npy', data_array)

In [ ]:
#process_data.write_to_hdf5(df, path_save)